In [11]:
#          ----------------------Table of Contents------------------------

# 1. Imported Data set.
# 2. Considering any security implications.
# 3. Comparing customer behavior in different geographic areas. 
# 4. Creating an exclusion flag for low-activity customers.
# 5. Creating a profiling variable. 
# 6. Showing the distribution of profiles. 
# 7. Aggregating measures on a customer-profile level for usage frequency and expenditure.
# 8. Comparing your customer profiles with regions and departments.
# 9. Produsing charts to illustrate the results. 
# 10. Export dataframe. 

In [12]:
# Importing libraries 
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [13]:
# Importing dataframe
path = r'C:\Users\ztrim\06-2025 Instacart Basket Analysis'

In [14]:
path

'C:\\Users\\ztrim\\06-2025 Instacart Basket Analysis'

In [15]:
# Import the merged dataset from 4.9  
df_merged = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'merged_dataframe_4_9.pkl'))

In [16]:
# 2. Considering any security implications.
df_merged_anonymized = df_merged.copy()
df_merged_anonymized = df_merged_anonymized.drop(columns=['first_name', 'surname', 'user_id'])
df_merged_anonymized.head()

,Unnamed: 0.1,Unnamed: 0_x,order_id,customers_current_order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,ord_regularity_median,frequency_flag,gender,state,age,date_joined,num_dependents,fam_status,income,merge_status
0,0,0,2539329,1,2,8,NaN,196,1,0,...,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
1,0,0,2539329,1,2,8,NaN,14084,2,0,...,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
2,0,0,2539329,1,2,8,NaN,12427,3,0,...,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
3,0,0,2539329,1,2,8,NaN,26088,4,0,...,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both
4,0,0,2539329,1,2,8,NaN,26405,5,0,...,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both


In [17]:
# 3. Comparing customer behavior in different geographic areas. 
# 3.1 Defining regions
northeast = [
    'Maine', 'New Hampshire', 'Vermont', 'Massachusetts', 'Rhode Island', 'Connecticut',
    'New York', 'New Jersey', 'Pennsylvania'
]

midwest = [
    'Ohio', 'Michigan', 'Indiana', 'Wisconsin', 'Illinois', 'Minnesota', 'Iowa',
    'Missouri', 'North Dakota', 'South Dakota', 'Nebraska', 'Kansas'
]

south = [
    'Delaware', 'Maryland', 'District of Columbia', 'Virginia', 'West Virginia',
    'North Carolina', 'South Carolina', 'Georgia', 'Florida', 'Kentucky', 'Tennessee',
    'Mississippi', 'Alabama', 'Oklahoma', 'Texas', 'Arkansas', 'Louisiana'
]

west = [
    'Idaho', 'Montana', 'Wyoming', 'Nevada', 'Utah', 'Colorado', 'Arizona',
    'New Mexico', 'Alaska', 'Washington', 'Oregon', 'California', 'Hawaii'
]

In [18]:
# 3.2 Assigning regions 
def assign_region(state):
    if state in northeast:
        return 'Northeast'
    elif state in midwest:
        return 'Midwest'
    elif state in south:
        return 'South'
    elif state in west:
        return 'West'
    else:
        return 'Other'

In [19]:
# 3.3 Applying function to create region column 
# Apply function
df_merged_anonymized['region'] = df_merged_anonymized['state'].apply(assign_region)

In [10]:
# 3.3 Checking the dataset (Adds a region column based on the state field)
df_merged_anonymized.head()

,Unnamed: 0.1,Unnamed: 0_x,order_id,customers_current_order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,frequency_flag,gender,state,age,date_joined,num_dependents,fam_status,income,merge_status,region
0,0,0,2539329,1,2,8,NaN,196,1,0,...,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both,Other
1,0,0,2539329,1,2,8,NaN,14084,2,0,...,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both,Other
2,0,0,2539329,1,2,8,NaN,12427,3,0,...,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both,Other
3,0,0,2539329,1,2,8,NaN,26088,4,0,...,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both,Other
4,0,0,2539329,1,2,8,NaN,26405,5,0,...,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423,both,Other


In [20]:
# 3.4 Determine differences in spending habits (Crosstab of high vs. low spenders by region. Useful for comparison)
region_spending = pd.crosstab(df_merged_anonymized['region'], df_merged_anonymized['spending_flag'])

In [21]:
# 3.4 check the output
region_spending.head()

spending_flag,High spender,Low spender
region,,
Midwest,155975,7441350
Northeast,108225,5614511
South,209691,10582194
West,160354,8132559


In [22]:
# 4. Creating an exclusion flag for low-activity customers (Shows how many customers have fewer than 5 orders)
df_merged_anonymized['exclusion_flag'] = df_merged_anonymized['max_order'] < 5
df_merged_anonymized['exclusion_flag'].value_counts()

exclusion_flag
False    30964564
True      1440295
Name: count, dtype: int64

## Clearing up memory before I continue 

In [23]:
df_merged_anonymized['order_id'] = df_merged_anonymized['order_id'].astype('int32')

In [24]:
df_merged_anonymized['orders_day_of_week'] = df_merged_anonymized['orders_day_of_week'].astype('int8')

In [25]:
df_merged_anonymized['days_since_prior_order'] = df_merged_anonymized['days_since_prior_order'].astype('float16')

In [26]:
df_merged_anonymized['customers_current_order_number'] = df_merged_anonymized['customers_current_order_number'].astype('int8')

In [27]:
df_merged_anonymized['order_hour_of_day'] = df_merged_anonymized['order_hour_of_day'].astype('int8')

In [28]:
# 4 Filtering active customers
df_active_anonymized = df_merged_anonymized[df_merged_anonymized['exclusion_flag'] == False]

In [29]:
df_active_anonymized.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30964564 entries, 0 to 32404858
Data columns (total 36 columns):
 #   Column                          Dtype   
---  ------                          -----   
 0   Unnamed: 0.1                    int64   
 1   Unnamed: 0_x                    int64   
 2   order_id                        int32   
 3   customers_current_order_number  int8    
 4   orders_day_of_week              int8    
 5   order_hour_of_day               int8    
 6   days_since_prior_order          float16 
 7   product_id                      int64   
 8   add_to_cart_order               int64   
 9   reordered                       int64   
 10  _merge                          category
 11  Unnamed: 0_y                    int64   
 12  product_name                    object  
 13  aisle_id                        int64   
 14  department_id                   int64   
 15  prices                          float64 
 16  match                           category
 17  busiest_day

In [30]:
# Export the active_anonymized dataframe (as per instructions in Part 1 Step 4) 
df_active_anonymized.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'active_anonymized_step_4'))

In [51]:
# Export df_merged_anonymized dataframe 
df_merged_anonymized.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'merged_anonymized.dataframe_4_10.pkl'))

In [57]:
df_merged_anonymized.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32404859 entries, 0 to 32404858
Data columns (total 36 columns):
 #   Column                          Dtype   
---  ------                          -----   
 0   Unnamed: 0.1                    int64   
 1   Unnamed: 0_x                    int64   
 2   order_id                        int32   
 3   customers_current_order_number  int8    
 4   orders_day_of_week              int8    
 5   order_hour_of_day               int8    
 6   days_since_prior_order          float16 
 7   product_id                      int64   
 8   add_to_cart_order               int64   
 9   reordered                       int64   
 10  _merge                          category
 11  Unnamed: 0_y                    int64   
 12  product_name                    object  
 13  aisle_id                        int64   
 14  department_id                   int64   
 15  prices                          float64 
 16  match                           category
 17  busies

In [2]:
df_merged_anonymized.describe()

NameError: name 'df_merged_anonymized' is not defined